# Project 3: Neo4j Code Portion

University of California, Berkeley

Master of Information and Data Science (MIDS) program

w205 - Fundamentals of Data Engineering

## Group: Kara Christensen, Negin Djalali, Gray Selby, Yidai Yao

### Start with setup code (importing libraries and setting up connections

In [1]:
import neo4j

import csv
import json
import openpyxl

import math
import numpy as np
import pandas as pd

import psycopg2


#### Postgres

In [ ]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [ ]:
cursor = connection.cursor()

#### Neo4j

In [32]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [33]:
session = driver.session(database="neo4j")

### Load in Ridership Data from BART (IN PROGRESS, NOT CURRENTLY USED)

In [9]:
def my_export_excel_work_book_sheet_to_csv(excel_file_name, sheet_name, csv_file_name):
    "open the excel file, find the sheet_name, export to csv file "
    
    work_book = openpyxl.load_workbook(excel_file_name)
    
    work_sheet = work_book[sheet_name]
                                       
    print("\nExcel Workbook:", excel_file_name)
          
    print("\nWork Sheet:", sheet_name)
    
    print("\nNumber of Columns:", work_sheet.max_column)
          
    print("\nNumber of Rows:", work_sheet.max_row)
    
    work_sheet_list = []
    
    for i in range(1,work_sheet.max_row+1):
        
        row_list = []
        
        for j in range(1,work_sheet.max_column+1):
            
            row_list.append(work_sheet.cell(row = i, column = j).value)
            
        work_sheet_list.append(row_list)
    
    print("\nExtracting Work Sheet", sheet_name, "to", csv_file_name, "\n")
    
    f = open(csv_file_name, "w")    
    writer = csv.writer(f)
    writer.writerows(work_sheet_list)
    f.close()

In [13]:
my_export_excel_work_book_sheet_to_csv('Ridership_202309.xlsx', 'Total Trips OD', 'Sept2023Riders.csv')
my_export_excel_work_book_sheet_to_csv('Ridership_202310.xlsx', 'Total Trips OD', 'Oct2023Riders.csv')
my_export_excel_work_book_sheet_to_csv('Ridership_202311.xlsx', 'Total Trips OD', 'Nov2023Riders.csv')


Excel Workbook: Ridership_202309.xlsx

Work Sheet: Total Trips OD

Number of Columns: 52

Number of Rows: 52

Extracting Work Sheet Total Trips OD to Sept2023Riders.csv 


Excel Workbook: Ridership_202310.xlsx

Work Sheet: Total Trips OD

Number of Columns: 52

Number of Rows: 52

Extracting Work Sheet Total Trips OD to Oct2023Riders.csv 


Excel Workbook: Ridership_202311.xlsx

Work Sheet: Total Trips OD

Number of Columns: 52

Number of Rows: 52

Extracting Work Sheet Total Trips OD to Nov2023Riders.csv 



In [16]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [28]:
#
# drop all the temp tables in the foreign key order
#

connection.rollback()

query = """

drop table if exists septRiders;
drop table if exists octRiders;
drop table if exists novRiders;

"""

cursor.execute(query)

connection.commit()

In [29]:
#
# create all the temp tables in the foreign key order
#

connection.rollback()

query = """

create table septRiders (
    Station varchar(32), 
    "RM" numeric(6), 
    EN numeric(6), 
    EP numeric(6), 
    NB numeric(6), 
    BK numeric(6), 
    "AS" numeric(6), 
    MA numeric(6), 
    "19" numeric(6), 
    "12" numeric(6), 
    LM numeric(6), 
    FV numeric(6), 
    CL numeric(6), 
    SL numeric(6), 
    BF numeric(6), 
    HY numeric(6), 
    SH numeric(6), 
    UC numeric(6), 
    FM numeric(6), 
    CN numeric(6), 
    PH numeric(6), 
    WC numeric(6), 
    LF numeric(6), 
    "OR" numeric(6), 
    RR numeric(6), 
    OW numeric(6), 
    EM numeric(6), 
    MT numeric(6), 
    PL numeric(6), 
    CC numeric(6), 
    "16" numeric(6), 
    "24" numeric(6), 
    GP numeric(6), 
    BP numeric(6), 
    DC numeric(6), 
    CM numeric(6), 
    CV numeric(6), 
    ED numeric(6), 
    NC numeric(6), 
    WP numeric(6), 
    SS numeric(6), 
    SB numeric(6), 
    SO numeric(6), 
    MB numeric(6), 
    WD numeric(6), 
    OA numeric(6), 
    WS numeric(6), 
    ML numeric(6), 
    BE numeric(6), 
    PC numeric(6), 
    AN numeric(6), 
    Exits numeric(7),
    primary key (Station)
);

create table octRiders (
    Station varchar(32), 
    RM numeric(6), 
    EN numeric(6), 
    EP numeric(6), 
    NB numeric(6), 
    BK numeric(6), 
    "AS" numeric(6), 
    MA numeric(6), 
    "19" numeric(6), 
    "12" numeric(6), 
    LM numeric(6), 
    FV numeric(6), 
    CL numeric(6), 
    SL numeric(6), 
    BF numeric(6), 
    HY numeric(6), 
    SH numeric(6), 
    UC numeric(6), 
    FM numeric(6), 
    CN numeric(6), 
    PH numeric(6), 
    WC numeric(6), 
    LF numeric(6), 
    "OR" numeric(6), 
    RR numeric(6), 
    OW numeric(6), 
    EM numeric(6), 
    MT numeric(6), 
    PL numeric(6), 
    CC numeric(6), 
    "16" numeric(6), 
    "24" numeric(6), 
    GP numeric(6), 
    BP numeric(6), 
    DC numeric(6), 
    CM numeric(6), 
    CV numeric(6), 
    ED numeric(6), 
    NC numeric(6), 
    WP numeric(6), 
    SS numeric(6), 
    SB numeric(6), 
    SO numeric(6), 
    MB numeric(6), 
    WD numeric(6), 
    OA numeric(6), 
    WS numeric(6), 
    ML numeric(6), 
    BE numeric(6), 
    PC numeric(6), 
    AN numeric(6), 
    Exits numeric(7),
    primary key (Station)
);

create table NovRiders (
    Station varchar(32), 
    RM numeric(6), 
    EN numeric(6), 
    EP numeric(6), 
    NB numeric(6), 
    BK numeric(6), 
    "AS" numeric(6), 
    MA numeric(6), 
    "19" numeric(6), 
    "12" numeric(6), 
    LM numeric(6), 
    FV numeric(6), 
    CL numeric(6), 
    SL numeric(6), 
    BF numeric(6), 
    HY numeric(6), 
    SH numeric(6), 
    UC numeric(6), 
    FM numeric(6), 
    CN numeric(6), 
    PH numeric(6), 
    WC numeric(6), 
    LF numeric(6), 
    "OR" numeric(6), 
    RR numeric(6), 
    OW numeric(6), 
    EM numeric(6), 
    MT numeric(6), 
    PL numeric(6), 
    CC numeric(6), 
    "16" numeric(6), 
    "24" numeric(6), 
    GP numeric(6), 
    BP numeric(6), 
    DC numeric(6), 
    CM numeric(6), 
    CV numeric(6), 
    ED numeric(6), 
    NC numeric(6), 
    WP numeric(6), 
    SS numeric(6), 
    SB numeric(6), 
    SO numeric(6), 
    MB numeric(6), 
    WD numeric(6), 
    OA numeric(6), 
    WS numeric(6), 
    ML numeric(6), 
    BE numeric(6), 
    PC numeric(6), 
    AN numeric(6), 
    Exits numeric(7),
    primary key (Station)
);

"""

cursor.execute(query)

connection.commit()

In [30]:
connection.rollback()

query = """

copy septRiders
from '/user/projects/ChristensenDjalaliSelbyYaoProject3/code/Sept2023Riders.csv' delimiter ',' NULL '' csv header;

copy octRiders
from '/user/projects/ChristensenDjalaliSelbyYaoProject3/code/Oct2023Riders.csv' delimiter ',' NULL '' csv header;

copy novRiders
from '/user/projects/ChristensenDjalaliSelbyYaoProject3/code/Nov2023Riders.csv' delimiter ',' NULL '' csv header;




"""

cursor.execute(query)

connection.commit()

In [31]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select * 
from septRiders;

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,rm,en,ep,nb,bk,AS,ma,19,12,...,so,mb,wd,oa,ws,ml,be,pc,an,exits
0,RM,485,1927,1768,994,7022,1359,2168,1618,1878,...,1181,383,77,251,299,178,266,144,274,57781
1,EN,2022,514,1797,1250,10827,1735,2859,3593,4698,...,1684,772,181,674,491,339,560,80,220,93389
2,EP,1827,1635,289,835,8263,1192,1719,2368,2563,...,1317,298,93,458,191,186,278,50,68,52223
3,NB,1127,1080,855,320,2279,803,1172,1895,1881,...,1584,367,51,526,194,188,246,51,79,44240
4,BK,7984,11378,8742,2688,1083,4049,4713,5935,6379,...,2872,1757,534,1317,832,1703,2523,245,589,145031
5,AS,1497,1708,1265,800,3883,335,1093,2114,2149,...,1153,311,48,393,199,276,359,82,161,50881
6,MA,2562,2758,1643,1132,4519,1062,510,2495,2395,...,1890,463,252,493,482,510,652,537,1605,92764
7,19,1731,3733,2404,1971,5588,2222,2518,541,542,...,1771,866,561,521,772,896,1059,412,1367,115095
8,12,2058,4683,2517,1903,5714,2129,2408,530,630,...,2606,894,400,662,676,724,1022,482,1892,120430
9,LM,596,987,625,607,2497,737,768,422,407,...,447,133,1143,464,943,651,816,129,405,73259


### Create the Neo4j Bart Map

#### First, the supplemental functions we will use

In [34]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [35]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [36]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")


In [37]:
def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)

In [38]:
def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [39]:
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)

#### Next, creating the actual graph itself

In [49]:
my_neo4j_wipe_out_database()

In [50]:
## This section adds the stations (a departure and arrival node for each one)
connection.rollback()

query = """

select station
from stations
order by station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    
    my_neo4j_create_node('depart ' + station)
    my_neo4j_create_node('arrive ' + station)
    
    
    
## This section adds the lines, and a relationship from each line at a station to the departure and arrival nodes  
connection.rollback()

query = """

select 
    station,
    line
from lines
order by station, line

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    line = row[1]
    
    my_neo4j_create_node(line + ' ' + station)
    
    my_neo4j_create_relationship_one_way('depart ' + station,line + ' ' + station,0)
    my_neo4j_create_relationship_one_way(line + ' ' + station,'arrive ' + station,0)
    
    
## This section adds the transfers between lines at each station
connection.rollback()

query = """

select
    station,
    from1.line as from_line,
    to1.line as to_line,
    transfer_time
from lines as from1
    join lines as to1
        using(station)
    join stations
        using(station)
        
where from1.line != to1.line
    
order by 1, 2, 3

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    from_line = row[1]
    to_line = row[2]
    transfer_time = row[3]
    
    my_neo4j_create_relationship_one_way(from_line + ' ' + station,to_line + ' ' + station,int(transfer_time))
    
    
## This section adds the relationship between adjacent stations on the same line
connection.rollback()

query = """

select
    from1.line,
    from1.station as from_station,
    to1.station as to_station,
    travel_time
from lines as from1
    join lines as to1
        on from1.line = to1.line
        and from1.sequence + 1 = to1.sequence
    join travel_times
        on (from1.station = station_1 and to1.station = station_2)
            or (from1.station = station_2 and to1.station = station_1)
    

order by 1, 2, 3


"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    line = row[0]
    from_station = row[1]
    to_station = row[2]
    travel_time = row[3]
    
    my_neo4j_create_relationship_two_way(line + ' ' + from_station, line + ' ' + to_station, int(travel_time))

In [51]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 652
-------------------------
